Data ingestion 


In [0]:
iot_raw_df = spark.read \
    .option("recursiveFileLookup", "true") \
    .json("/Volumes/workspace/default/iot_dataa/IOT Device Status Messages/")


schema exploration 

In [0]:
iot_raw_df.printSchema()
iot_raw_df.show(5, truncate=False)


root
 |-- Body: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- mnsn: string (nullable = true)
 |    |-- scheduleName: string (nullable = true)
 |    |-- smsProperties: struct (nullable = true)
 |    |    |-- device: struct (nullable = true)
 |    |    |    |-- address: string (nullable = true)
 |    |    |    |-- capability: struct (nullable = true)
 |    |    |    |    |-- function: struct (nullable = true)
 |    |    |    |    |    |-- deviceCloning: string (nullable = true)
 |    |    |    |    |    |-- fss: string (nullable = true)
 |    |    |    |    |    |-- fssDataCloning: struct (nullable = true)
 |    |    |    |    |    |    |-- available: string (nullable = true)
 |    |    |    |    |    |    |-- version: string (nullable = true)
 |    |    |    |    |    |-- fwUpdateOTA: struct (nullable = true)
 |    |    |    |    |    |    |-- available: string (nullable = true)
 |    |    |    |    |    |    |-- version: string (nullable = true)
 |    |   

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp


transformation 

In [0]:
iot_transformed_df = (
    iot_raw_df
    .select(
        col("Body.mnsn").alias("mnsn"),
        col("Body.smsProperties.device.errorLevel").alias("statusCode"),
        col("Body.smsProperties.device.statusRawValue").alias("statusRawValue")
    )
    .withColumn("_id", col("mnsn"))
    .withColumn("equipmentId", col("mnsn"))
    .withColumn("serialNumber", col("mnsn"))
    .withColumn("type", lit("mfp"))
    .withColumn("attribute", lit("assigned"))
    .withColumn("createTimestamp", current_timestamp().cast("long"))
    .withColumn("lastStatusUpdate", current_timestamp().cast("long"))
)


deduplication


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

window_spec = Window.partitionBy("mnsn").orderBy(desc("lastStatusUpdate"))

iot_dedup_df = (
    iot_transformed_df
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)


In [0]:
iot_dedup_df.printSchema()


root
 |-- mnsn: string (nullable = true)
 |-- statusCode: string (nullable = true)
 |-- statusRawValue: string (nullable = true)
 |-- _id: string (nullable = true)
 |-- equipmentId: string (nullable = true)
 |-- serialNumber: string (nullable = true)
 |-- type: string (nullable = false)
 |-- attribute: string (nullable = false)
 |-- createTimestamp: long (nullable = false)
 |-- lastStatusUpdate: long (nullable = false)



In [0]:
iot_dedup_df.show(5, truncate=False)


+-----------------------------------+----------+--------------+-----------------------------------+-----------------------------------+-----------------------------------+----+---------+---------------+----------------+
|mnsn                               |statusCode|statusRawValue|_id                                |equipmentId                        |serialNumber                       |type|attribute|createTimestamp|lastStatusUpdate|
+-----------------------------------+----------+--------------+-----------------------------------+-----------------------------------+-----------------------------------+----+---------+---------------+----------------+
|mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==|NULL      |5302048       |mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==|mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==|mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==|mfp |assigned |1766657488     |1766657488      |
|mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==|NULL      |3308192       |mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==|mn=QlA1MEMyNg==:sn=Mj

In [0]:
from pyspark.sql.functions import input_file_name


In [0]:
iot_with_file_time = iot_raw_df.withColumn(
    "fileTime",
    input_file_name()
)


delta table creation

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS device_latest (
    _id STRING,
    equipmentId STRING,
    serialNumber STRING,
    type STRING,
    attribute STRING,
    statusCode INT,
    statusRawValue STRING,
    createTimestamp LONG,
    lastStatusUpdate LONG
)
USING DELTA
""")


DataFrame[]

In [0]:
spark.sql("DESCRIBE DETAIL device_latest").show(truncate=False)


+------+------------------------------------+-------------------------------+-----------+--------+-----------------------+-------------------+----------------+-----------------+--------+-----------+------------------------------------------------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+-------------+
|format|id                                  |name                           |description|location|createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                                                                    |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                                                     |clusterByAuto|
+------+------------------------------------+-------------------------------+-----------+--------+-----------------------+--------------

Merge

In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "device_latest")

(
    delta_table.alias("target")
    .merge(
        iot_dedup_df.alias("source"),
        "target._id = source._id"
    )
    .whenMatchedUpdate(set={
        "createTimestamp": "source.createTimestamp",
        "equipmentId": "source.equipmentId",
        "type": "source.type",
        "attribute": "source.attribute"
    })
    .whenNotMatchedInsert(values={
        "_id": "source._id",
        "equipmentId": "source.equipmentId",
        "type": "source.type",
        "attribute": "source.attribute",
        "createTimestamp": "source.createTimestamp"
    })
    .execute()
)


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

validating

In [0]:
spark.table("device_latest").show(20, truncate=False)


+-----------------------------------+---------------+----+---------+------------------------------------+------+-------------------+------------------------------------+--------+----------+----------+----------+----------+----------+------------+-----------------------------------+------+-----------------+--------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
spark.table("device_latest") \
    .groupBy("_id") \
    .count() \
    .filter("count > 1") \
    .show()


+---+-----+
|_id|count|
+---+-----+
+---+-----+



In [0]:
spark.table("device_latest").show(
    n=5,
    truncate=False,
    vertical=True
)


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
spark.table("device_latest") \
    .select(
        "_id",
        "type",
        "attribute",
        "equipmentId",
        "createTimestamp"
    ) \
    .show(truncate=False)


+-----------------------------------+----+---------+-----------------------------------+---------------+
|_id                                |type|attribute|equipmentId                        |createTimestamp|
+-----------------------------------+----+---------+-----------------------------------+---------------+
|mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==|mfp |assigned |mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==|1766657779     |
|mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==|mfp |assigned |mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==|1766657779     |
|mn=QlA1MEMyNg==:sn=MzMzMzMzMzMyMw==|mfp |assigned |mn=QlA1MEMyNg==:sn=MzMzMzMzMzMyMw==|1766657779     |
|mn=TVg1MDcwTg==:sn=STU0NjQz        |mfp |assigned |mn=TVg1MDcwTg==:sn=STU0NjQz        |1766657779     |
|mn=TVgzNTYwTg==:sn=NzUwNjkzMjYwMA==|mfp |assigned |mn=TVgzNTYwTg==:sn=NzUwNjkzMjYwMA==|1766657779     |
+-----------------------------------+----+---------+-----------------------------------+---------------+



DISPLAYING NULL COLUMNS ALSO 

In [0]:
target_df = spark.table("device_latest")
target_df.printSchema()


root
 |-- _id: string (nullable = true)
 |-- createTimestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- attribute: string (nullable = true)
 |-- relatedGroupId: string (nullable = true)
 |-- tagIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- communicationStatus: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- dealerId: string (nullable = true)
 |-- L1DealerId: string (nullable = true)
 |-- L2DealerId: string (nullable = true)
 |-- L3DealerId: string (nullable = true)
 |-- divisionId: string (nullable = true)
 |-- onContract: boolean (nullable = true)
 |-- purchaseDate: long (nullable = true)
 |-- equipmentId: string (nullable = true)
 |-- isSNAP: boolean (nullable = true)
 |-- servicingDealerId: string (nullable = true)
 |-- SAP_name: string (nullable = true)
 |-- SAP_location: string (nullable = true)
 |-- deviceGeneral: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- capabil

In [0]:
from pyspark.sql.functions import lit, col


In [0]:
aligned_df = spark.table("device_latest").select(

    # --- Identifiers ---
    col("_id"),
    col("createTimestamp"),
    col("type"),
    col("attribute"),

    # --- These exist in table but not in JSON → NULL ---
    col("relatedGroupId"),
    col("tagIds"),
    col("communicationStatus"),
    col("customerId"),
    col("dealerId"),
    col("L1DealerId"),
    col("L2DealerId"),
    col("L3DealerId"),
    col("divisionId"),
    col("onContract"),
    col("purchaseDate"),

    # --- Mapped ---
    col("equipmentId"),

    # --- More unmatched → NULL ---
    col("isSNAP"),
    col("servicingDealerId"),
    col("SAP_name"),
    col("SAP_location"),

    # --- Nested objects (already structured) ---
    col("deviceGeneral"),
    col("printerDevice"),

    # --- Optional ---
    col("counter")
)


In [0]:
aligned_df.show(truncate=False, vertical=True)


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
final_df = spark.table("device_latest") \
    .select(
        "_id",
        "createTimestamp",
        "relatedGroupId",
        "tagIds",
        "communicationStatus",
        "customerId",
        "dealerId",
        "type",
        "attribute",
        "equipmentId"
    )

display(final_df)


_id,createTimestamp,relatedGroupId,tagIds,communicationStatus,customerId,dealerId,type,attribute,equipmentId
mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==,1766657779,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,mfp,assigned,mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==
mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==,1766657779,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,mfp,assigned,mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==
mn=QlA1MEMyNg==:sn=MzMzMzMzMzMyMw==,1766657779,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,mfp,assigned,mn=QlA1MEMyNg==:sn=MzMzMzMzMzMyMw==
mn=TVg1MDcwTg==:sn=STU0NjQz,1766657779,2344cb97-4ebe-4412-8a8c-64730003681c,null,90,ea387dfe-357c-48b7-818a-e01063c184f5,281083,mfp,assigned,mn=TVg1MDcwTg==:sn=STU0NjQz
mn=TVgzNTYwTg==:sn=NzUwNjkzMjYwMA==,1766657779,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,mfp,assigned,mn=TVgzNTYwTg==:sn=NzUwNjkzMjYwMA==


checking final no duplicates

In [0]:
display(
    spark.table("device_latest")
         .groupBy("_id")
         .count()
         .filter("count > 1")
)


_id,count
